In [ ]:
#Esta abordagem é filtragem colaborativa baseada em itens (item-based collaborative filtering)
!pip install pandas numpy scipy scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

#Configurar diretório e carregar dados
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

dataset_dir = "ml-latest-small"

#Carregar avaliações
ratings = pd.read_csv(f"{dataset_dir}/ratings.csv")  #userId, movieId, rating, timestamp

#Carregar filmes
movies = pd.read_csv(f"{dataset_dir}/movies.csv")    #movieId, title, genres

print("Dados carregados!")
print(f"Total de avaliações: {len(ratings)}")
print(f"Total de filmes: {len(movies)}")

In [ ]:
#Mapear IDs para índices e criar matriz esparsa

#O objetivo aqui é transformar os IDs originais (que não são sequenciais)
#em índices contínuos (0, 1, 2, ...) — algo necessário para criar uma matriz.

#Criaremos dicionários de mapeamento de IDs para índices
#Cada linha em ratings representa uma avaliação feita por um usuário a um filme
#Porém os IDs originais em ratings['userId'] e ratings['movieId'] podem não ser sequenciais.
#Para criar matriz users x movies precisamos índices 0..N-1.

#.unique() retorna um array NumPy contendo apenas os valores distintos de uma coluna, na ordem em que aparecem
#enumerate() transforma uma sequência em pares (índice, valor)
'''Se ratings.csv contiver:
   | userId | movieId | rating |
   | ------ | ------- | ------ |
   | 10     | 5       | 3.5    |
   | 42     | 2       | 4.0    |
   | 10     | 7       | 2.0    |
   Então user_mapper = {10: 0, 42: 1}, movie_mapper = {5: 0, 2: 1, 7: 2}
   Usuário 10 vira linha 0, Usuário 42 vira linha 1
   Filme 5 vira coluna 0, Filme 2 vira coluna 1, Filme 7 vira coluna 2
   Matriz esparsa ficará com shape (2, 3) — 2 usuários × 3 filmes.
'''

#Cria mapeamento de cada userId único para um índice sequencial
user_mapper = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#Cria mapeamento de cada movieId único para um índice sequencial
movie_mapper = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}

#Mapeamento inverso (para retornar IDs originais)
movie_inv_mapper = {i: mid for mid, i in movie_mapper.items()}
user_inv_mapper = {i: uid for uid, i in user_mapper.items()}

#Mapear IDs originais para índices, adiciona colunas com os índices internos
ratings['user_idx'] = ratings['userId'].map(user_mapper)
ratings['movie_idx'] = ratings['movieId'].map(movie_mapper)

#NOVO: normalização das notas + ponderação por similaridade
#Usuários têm padrões de avaliação diferentes: uns dão sempre 5 estrelas, outros quase nunca passam de 3.5
#Ao calcular a similaridade, devemos centralizar (subtrair a média de cada usuário)
#Isso remove o “viés do usuário” e aumenta significativamente a qualidade preditiva do modelo
#O modelo se aproxima mais de um Collaborative Filtering baseado em correlação de Pearson

#Calcular média das notas por usuário
user_mean = ratings.groupby('userId')['rating'].mean()
#Subtrair a média de cada usuário
ratings['rating_norm'] = ratings.apply(lambda x: x['rating'] - user_mean.loc[x['userId']], axis=1)

#Criar matriz esparsa (usuários x filmes)
ratings_sparse = csr_matrix(
    (ratings['rating_norm'], (ratings['user_idx'], ratings['movie_idx']))
)

'''Com o exemplo usado antes:
   user_mapper = {10: 0, 42: 1}
   movie_mapper = {5: 0, 2: 1, 7: 2}
   A matriz de usuários × filmes fica:        5     2     7
                                       U10 [ 3.5   0.0   2.0 ]
                                       U42 [ 0.0   4.0   0.0 ]
   Então a representação esparsa (armazenando só valores != 0) fica:
            (0,0)  3.5 --> user 10, movie 5
            (0,2)  2.0 --> user 10, movie 7
            (1,1)  4.0 --> user 42, movie 2
'''

print("Matriz esparsa criada!")
print(f"Shape da matriz: {ratings_sparse.shape}")

In [ ]:
#Calcular similaridade entre filmes

#.T = operador de transposição. ratings_sparse tem usuários nas linhas e filmes nas colunas
#Transposição é feita para ter filmes nas linhas (necessário para filme-filme similarity)
#dense_output=False mantém saída esparsa.
movie_similarity = cosine_similarity(ratings_sparse.T, dense_output=False)

print("Matriz de similaridade entre filmes calculada!")

In [76]:
#Seleciona os Top-K vizinhos por item, ignorando similaridades negativas

K = 100  #pode ser alterado livremente

#Converte a matriz de similaridade para formato CSR para operações rápidas
movie_similarity_csr = movie_similarity.tocsr()

#Inicializa lista para armazenar os índices e valores de similaridade dos Top-K vizinhos
indices_topk = []
values_topk = []

for i in range(movie_similarity_csr.shape[0]):
    #Extrai a linha i (similaridade do filme i com todos os outros)
    row = movie_similarity_csr.getrow(i).toarray().ravel()

    #Zera similaridades negativas para ignorá-las
    row[row < 0] = 0

    #Remove o próprio item (autossimilaridade = 1)
    row[i] = 0

    #Seleciona apenas os índices dos K maiores valores positivos
    topk_idx = np.argsort(row)[-K:][::-1]  #pega os maiores K
    topk_values = row[topk_idx]

    #Remove zeros do resultado (caso o item tenha < K positivos)
    valid_mask = topk_values > 0
    topk_idx = topk_idx[valid_mask]
    topk_values = topk_values[valid_mask]

    indices_topk.append(topk_idx)
    values_topk.append(topk_values)

#Agora, indices_topk[i] contém os índices dos vizinhos positivos mais similares ao item i
#e values_topk[i] contém os respectivos valores de similaridade

In [77]:
#Função para recomendar filmes similares
#Retorna os top_n filmes mais similares a um movie_id dado
#Usa apenas os vizinhos positivos mais similares do filme como base de recomendação -- ou seja, substitui a parte onde pega todos os scores pela lista indices_topk[movie_idx].
def recommend_movies(movie_id, top_n):
    #Verifica se o filme existe no mapeamento
    if movie_id not in movie_mapper:
        print(f"Filme {movie_id} não encontrado.")
        return []

    #Obtém o índice interno do filme
    movie_idx = movie_mapper[movie_id]

    #Obtém vizinhos mais similares pré-calculados (Top-K)
    topk_indices = indices_topk[movie_idx]
    topk_values = values_topk[movie_idx]

    #Cria uma série (índice = índice do filme, valor = similaridade)
    sim_series = pd.Series(topk_values, index=topk_indices)

    #Seleciona os Top N (ou menos, se houver menos que N vizinhos)
    top_indices = sim_series.sort_values(ascending=False).iloc[:top_n].index

    #Monta lista de recomendações com títulos e similaridades
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]                                   #Converte índice interno de volta para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]  #Busca título do filme
        score = sim_series[idx]                                       #Similaridade calculada
        recommendations.append((title, score))

    return recommendations

In [78]:
#Testar recomendações para um filme

#Exemplo: movieId = 1 (Toy Story 1995)
movie_id = 1
top_n=15
top_movies = recommend_movies(movie_id, top_n)

print(f"Top {top_n} filmes similares a '{movies.loc[movies.movieId==movie_id,'title'].values[0]}':")
for title, score in top_movies:
    print(f"{title} — similaridade {score:.3f}")

Top 15 filmes similares a 'Toy Story (1995)':
Toy Story 2 (1999) — similaridade 0.403
Toy Story 3 (2010) — similaridade 0.327
Aladdin (1992) — similaridade 0.327
Wallace & Gromit: The Wrong Trousers (1993) — similaridade 0.305
Back to the Future (1985) — similaridade 0.277
Incredibles, The (2004) — similaridade 0.275
Blazing Saddles (1974) — similaridade 0.272
Finding Nemo (2003) — similaridade 0.263
Ghostbusters (a.k.a. Ghost Busters) (1984) — similaridade 0.250
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) — similaridade 0.237
Lion King, The (1994) — similaridade 0.235
True Grit (1969) — similaridade 0.226
Hugo (2011) — similaridade 0.224
E.T. the Extra-Terrestrial (1982) — similaridade 0.224
Mary Poppins (1964) — similaridade 0.224


In [79]:
#Função para recomendar para um usuário
#Retorna recomendações para um usuário baseado em ratings passados e similaridade filme-filme
def recommend_for_user(user_id, top_n=5):
    #Obter índice do usuário
    if user_id not in user_mapper:
        print(f"Usuário {user_id} não encontrado.")
        return []

    #Converte user_id original para o índice interno da matriz
    user_idx = user_mapper[user_id]

    #Filmes avaliados pelo usuário (linha do usuário na matriz)
    #Converte a linha esparsa para array e achata para vetor 1D
    user_ratings = ratings_sparse[user_idx, :].toarray().flatten()

    #Pega os índices dos filmes avaliados (notas > 0)
    rated_indices = np.where(user_ratings > 0)[0]

    #Vetores para armazenar soma ponderada dos scores e soma das similaridades
    scores = np.zeros(ratings_sparse.shape[1])
    sim_sums = np.zeros(ratings_sparse.shape[1])

    '''Para cada filme avaliado, acumular score ponderado:
       - pega os vizinhos Top-K desse filme
       - multiplica a similaridade de cada vizinho pela nota do usuário
       - acumula as somas para depois normalizar'''
    for idx in rated_indices:
        sim_indices = indices_topk[idx]   # índices dos vizinhos Top-K
        sim_values = values_topk[idx]     # valores das similaridades Top-K
        rating = user_ratings[idx]        # nota (normalizada) que o usuário deu ao filme

        #Acumula soma ponderada e normalizador
        scores[sim_indices] += sim_values * rating
        sim_sums[sim_indices] += sim_values

    #Evita divisão por zero
    sim_sums[sim_sums == 0] = 1

    #Nota prevista normalizada = soma ponderada / soma das similaridades
    predicted_ratings = scores / sim_sums

    #---------IMPORTANTE: REVERTER NORMALIZAÇÃO PARA ESCALA ORIGINAL---------
    #user_avg = user_mean.loc[user_id]                 #média real do usuário
    #predicted_ratings = predicted_ratings + user_avg  #soma de volta à média

    #Limita à escala válida do MovieLens (0.5 a 5.0)
    #predicted_ratings = np.clip(predicted_ratings, 0.5, 5.0)

    #Remove filmes já avaliados (não queremos recomendá-los novamente)
    predicted_ratings[rated_indices] = 0

    #Seleciona os filmes com maiores notas previstas
    top_indices = np.argsort(predicted_ratings)[::-1][:top_n]

    #Monta a lista final de recomendações (título e nota prevista)
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]   # converte índice interno para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]
        recommendations.append((title, predicted_ratings[idx]))

    return recommendations

In [83]:
#Testar recomendações para um usuário

user_id = 18
top_user_movies = recommend_for_user(user_id, top_n=5)

print(f"Top 5 recomendações para o usuário {user_id}:")
for title, score in top_user_movies:
    print(f"{title} — score {score:.3f}")

Top 5 recomendações para o usuário 18:
Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965) — score 1.268
All Dogs Go to Heaven 2 (1996) — score 1.268
Boiling Point (1993) — score 1.268
Tender Mercies (1983) — score 1.268
Stepford Wives, The (1975) — score 1.268


## O que muda ao usar o Top-K
Na nova versão:
- predicted_ratings = scores / sim_sums;
- scores = soma das similaridades × nota do usuário;
- sim_sums = soma somente dos Top-K vizinhos relevantes;

E como estamos somando K vizinhos com similaridade real (ex: 0.7, 0.6, 0.5), o denominador (sim_sums) tende a ser pequeno (~3-6).

- Isso faz com que o resultado final (predicted_ratings) caia naturalmente para a escala das notas originais (0-5).
- Agora estamos efetivamente prevendo notas normalizadas, e não apenas acumulando força bruta.

Portanto, valores como 1.268 agora significam algo do tipo: “O modelo prevê que o usuário daria nota 1.27 para esse filme.”

**SEM** A NORMALIZAÇÃO DE VOLTA PARA A ESCALA ORIGINAL VÁLIDA DO MOVIELENS, o modelo está prevendo quanto acima ou abaixo da média o usuário tenderia a avaliar, não a nota final ainda.


Tomemos o usuário 18 de exemplo. Top 5 recomendações para o usuário 18 com k=100:
- Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965) — score 1.268
- All Dogs Go to Heaven 2 (1996) — score 1.268
- Boiling Point (1993) — score 1.268
- Tender Mercies (1983) — score 1.268
- Stepford Wives, The (1975) — score 1.268

O sistema está prevendo valores em torno de zero:
- 0 → filme médio para o usuário
- +1.0 → filme que ele tenderia a avaliar 1 ponto acima da média
- -1.0 → filme que ele tenderia a avaliar 1 ponto abaixo da média

Ao somar +3.73 (média pessoal dele), obtém-se previsões em torno de 3.73 ± 1, ou seja, na escala original (0.5–5.0).

In [81]:
#GEPETO: BLOCO GERADO PARA DIAGNÓSTICOS RÁPIDOS

#1) intervalo das notas no dataset original
print("ratings range:", ratings['rating'].min(), ratings['rating'].max())
print("ratings mean overall:", ratings['rating'].mean())

#2) verificar se você tem coluna normalizada (rating_norm) e se ratings_sparse usa ela
print("rating_norm present:", 'rating_norm' in ratings.columns)
#Se ratings_sparse foi reconstruída a partir de ratings['rating_norm'], verifique:
example_user = 442
print("média do usuário", example_user, ":", ratings.groupby('userId')['rating'].mean().loc[example_user])
print("user_ratings (sample) for user 18:", ratings_sparse[user_mapper[example_user], :].toarray().flatten()[ :10 ])

#3) estatísticas das similaridades Top-K para um filme exemplo
movie_idx = 2
vals = values_topk[movie_idx] if 'values_topk' in globals() else movie_similarity[movie_idx].toarray().flatten()
print("similarity stats (example movie): min, max, mean, sum:", np.min(vals), np.max(vals), np.mean(vals), np.sum(vals))

#4) rodar recommend_for_user e inspecionar sim_sums / predicted_ratings internamente
user_id = 442
user_idx = user_mapper[user_id]
user_ratings = ratings_sparse[user_idx,:].toarray().flatten()
rated_indices = np.where(user_ratings>0)[0]

#Compute sim_sums & scores as in your Top-K version
scores = np.zeros(ratings_sparse.shape[1])
sim_sums = np.zeros(ratings_sparse.shape[1])
for idx in rated_indices:
    sim_idx = indices_topk[idx]
    sim_val = values_topk[idx]
    scores[sim_idx] += sim_val * user_ratings[idx]
    sim_sums[sim_idx] += sim_val

print("sim_sums stats:", sim_sums.min(), np.percentile(sim_sums,25), np.median(sim_sums), np.mean(sim_sums), sim_sums.max())
pred = scores / np.where(sim_sums==0, 1, sim_sums)
print("predicted ratings stats:", pred.min(), np.percentile(pred,25), np.median(pred), np.mean(pred), pred.max())


ratings range: 0.5 5.0
ratings mean overall: 3.501556983616962
rating_norm present: True
média do usuário 442 : 1.275
user_ratings (sample) for user 18: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
similarity stats (example movie): min, max, mean, sum: 0.22215172888848148 0.3507745103783537 0.23575215303455174 23.575215303455174
sim_sums stats: 0.0 0.0 0.0 0.02777567959857245 0.757237232655326
predicted ratings stats: 0.0 0.0 0.0 0.053067961507589835 1.2250000000000003


In [70]:
#Descobrir o usuário com menor média de avaliações
lowest_user = user_mean.idxmin()       #retorna o userId com menor média
lowest_mean = user_mean.min()          #valor da menor média

print(f"Usuário com menor média: {lowest_user} (média = {lowest_mean:.3f})")

Usuário com menor média: 442 (média = 1.275)


In [71]:
print("Média do usuário:", user_mean.loc[442])
print("Número de filmes avaliados:", len(np.where(ratings_sparse[user_mapper[442], :].toarray().flatten() > 0)[0]))

#Avaliar dispersão das predições
pred = recommend_for_user(442, top_n=5)
print(pred)

Média do usuário: 1.275
Número de filmes avaliados: 9
[('Pom Poko (a.k.a. Raccoon War, The) (Heisei tanuki gassen pompoko) (1994)', np.float64(2.5)), ('Great Expectations (1946)', np.float64(2.5)), ('Doctor Who: The Time of the Doctor (2013)', np.float64(2.5)), ('Real Blonde, The (1997)', np.float64(2.5)), ('Talk to Her (Hable con Ella) (2002)', np.float64(2.5))]


Top 5 recomendações para o usuário 442, com k=100:
- Real Blonde, The (1997) — score 1.225
- Summer Place, A (1959) — score 1.225
- Beyond the Clouds (Al di là delle nuvole) (1996) — score 1.225
- Total Eclipse (1995) — score 1.225
- Final Analysis (1992) — score 1.225

Usuário 442 dá notas extremamente baixas (bem abaixo da média geral ≈ 3.5). Seu vetor de avaliações é muito esparso e de baixo valor, com 9 filmes avaliados.

Durante a predição, como normalizamos as notas (rating_norm = rating - user_mean), esse usuário 442 tem praticamente notas normalizadas próximas de 0.

Isso faz com que o acumulador
`scores[sim_indices] += sim_values * rating`
resulte em valores muito baixos ou quase nulos (porque rating ≈ 0 ou até negativo, e sim_values pequenas).

Na normalização, mesmo após dividir por sim_sums,
- `predicted_ratings = scores / sim_sums`
- `predicted_ratings += user_avg  #soma a média dele (1.275)`

as notas previstas se concentram em torno da média pessoal dele -- ou seja, tudo fica ≈ 1.27.

Mas como aplicamos np.clip(predicted_ratings, 0.5, 5.0) e np.where(...),
ainda podendo haver arredondamentos internos,
o valor final pode ser arredondado ou truncado para algo como 2.5,
especialmente se a matriz de similaridade for muito diluída (top-K pequeno ou falta de vizinhos relevantes).

Além disso, como avaliou poucos filmes (9) com notas baixas (média = 1.275), o modelo tem pouquíssima informação para inferir o gosto dele.

O algoritmo item-based tenta prever algo assim:
> “Se ele gostou de filme A, então talvez também goste de filme B semelhante.”

Mas… se ele não gostou de quase nada, não há ponto de referência positivo.
Logo, o modelo fica meio “sem chão”, e retorna um valor neutro (2.5) porque:
- é o centro da escala (entre 0.5 e 5.0),
- e o cálculo ponderado por similaridade tende a convergir para esse meio termo quando os pesos são fracos ou cancelam entre si.


Sim, escrevi esse textão todo só pra demonstrar como esse comportamento é normal. O modelo está apenas sendo honesto: “Eu não tenho informação suficiente para sugerir algo com confiança, então fico no meio da escala.”

O que observamos aqui é um dos fenômenos mais clássicos da área: o chamado “cold start” (início frio).